# A Non-Parametric Bayesian Method for Inferring Hidden Causes
<a href="http://cocosci.berkeley.edu/tom/papers/ibpuai.pdf">F., Griffiths, T.L., Ghahramani, Z., 2006.<br />
Presented at the Proceedings of the Conference on Uncertainty in Artificial Intelligence.</a>

### Reqirements
* #### You need to install module future, manual importing from \_\_future\_\_ is at your convenience
* #### For hdf data import you need pytables too which is not default installed with Anaconda

### Batch execution
* #### ```batch_animal=msaxxyy_z jupyter nbconvert Bayesian.ipynb --to=html --execute --ExecutePreprocessor.timeout=-1 --output=xxyy_z_report.html```

In [ ]:
#from future.utils import PY3
import future
from __future__ import (absolute_import, division,
                        print_function) #, unicode_literals)
import pandas as pd
import numpy as np
import time, os, warnings, imp, itertools
import IPython.display as disp
display = disp.display
import matplotlib as mpl, matplotlib.pyplot as plt
import scipy.stats as stats
zscore, describe = stats.mstats.zscore, stats.describe
import datetime
dt, td = datetime.datetime, datetime.timedelta

%matplotlib inline

In [ ]:
import ca_lib as la
imp.reload(la)

In [ ]:
from os import environ
batch_animal = environ.get('batch_animal', None)

## Load files

In [ ]:
basedir = '../_share/Losonczi/'

# Display database folders
display(os.listdir(basedir))

# Select animal
if batch_animal is None:
    #animal = 'msa0216_4'; FPS = 8
    #animal = 'msa0316_1'; FPS = 8
    #animal = 'msa0316_3'; FPS = 8
    animal = 'msa0316ag_1'; FPS = 8
    #animal = 'msa0915_1'; FPS = 30
    #animal = 'msa0915_2'; FPS = 30
    #animal = 'msa1215_1'; FPS = 30
else:
    FPS = None
    animal = batch_animal

print ('selecting',animal)

# List dir
mydir = os.path.join(basedir,animal)
os.listdir(mydir)

In [ ]:
# Available trials and ROIs
data = la.load_files(mydir)
processed = la.read_from_hdf('anidb_'+animal+'.h5', la.Bunch())
a_lick, b_lick, lick_threshold = processed['a_lick'], processed['b_lick'], processed['lick_threshold']
del processed

if (FPS is not None) and (data.FPS != FPS):
    warnings.warn('FPS indication might be wrong.')
print (data.raw.shape, '\n', data.trials, '\n', data.rois)

In [ ]:
behavior = {}
# Motivation
behavior['motivation'] = a_lick.iloc[:,0] > lick_threshold
# Fear
behavior['fear'] = a_lick.iloc[:,2] < lick_threshold
# Sensibility
behavior['sensibility'] = a_lick.iloc[:,3] < lick_threshold

In [ ]:
la.events

In [ ]:
340/data.FPS

In [ ]:
behavior['fear2'] = b_lick.iloc[:,8] < lick_threshold

In [ ]:
behavior['fear2'][behavior['fear2'] != behavior['fear']]

## Display

In [ ]:
# Post-Learning may repeat session_num therefore an additional index,
# day_num is created. See msa0316_1.
# It seems though that Pre-Learning and Learning treats session_num as documented.
display(data.experiment_traits.head())
display(data.experiment_traits[data.experiment_traits['day_leap']])

## Save for matlab

In [ ]:
np.array(data.experiment_traits.to_records()).shape

In [ ]:
np.ndarray == np.recarray

In [ ]:
import scipy.io as sio
def cellarray(df, index, dropna_axis=None, fillna_axis=None, fillna_method=None):
    '''Split a DataFrame with MultiIndex into a 1D cellarray'''
    import warnings
    ca = np.empty(shape=len(index), dtype=np.ndarray)
    for i, key in enumerate(index):
        tmp = df.loc[key]
        if dropna_axis is not None:
            tmp = tmp.dropna(axis=dropna_axis, how='all')
        if fillna_axis is not None:
            tmp = tmp.fillna(axis=fillna_axis, method=fillna_method).fillna(value=0)
        if (type(tmp.index) is pd.MultiIndex) or (type(tmp.columns) is pd.MultiIndex):
            warnings.warn('Matrix in a cell has multiindex.')
        ca[i] = tmp.values
        #print (tmp.shape, tmp.isnull().sum().sum())
    return ca

In [ ]:
### Select acitve ROIs and prepare them for output
# trial ID
ix = data.mirow.levels[0]
# fill rate of spiking (this is a full df)
mea = data.spike.unstack('time',fill_value=0).mean(axis=1)
# present in almost all frames of almost ll trials & active
keep = data.mask_roi & (mea>0.02)
# extract ROIs to keep
rois = data.mask_roi[keep].index
# statistics
print ('Keep %d ROIs out of %d.'%(len(rois),len(data.mask_roi)))
ref = pd.MultiIndex.from_product((data.mirow.levels[0],(rois)))
def prep(df, fill_value=None):
    '''Reindex (fill in the gaps) and split DataFrame to cellarray'''
    df = df.reindex(fill_value=None, index=ref, columns=data.icol)
    ret = cellarray(df, ix, dropna_axis=1, fillna_axis=0, fillna_method='ffill')
    return ret

w = {'transients': prep(data.spike),
     'filtered': prep(data.filtered),
     'raw': prep(data.raw),
     'mask': prep(data.mask),
     'trials':data.mirow.levels[0].values.astype(str),
     'rois':rois.values.astype(str),
     'frames':data.icol.values}
sio.savemat(animal+'.mat',w)

In [ ]:
include_w = True
# categoric feature: column, value
if include_w:
    cat_features = [('context', 'CS+'), ('context', 'CS-'), ('port', 'W+'), ('puffed', 'A+')]
else:
    cat_features = [('context', 'CS+'), ('context', 'CS-'), ('puffed', 'A+')]
# ordinal feature: column, list of allowed values
ord_features = []

## Bayesian inference

#### Prep data

In [ ]:
def create_features(cat_list, list_ord, data):
    '''Create the features to be learnt'''
    col = 0
    # features = pd.DataFrame(index=data.index, columns=[])
    features = []
    for column, criterion in cat_list:
        feat = data.loc[:,column] == criterion
        feat.name = '%d_%s' % (col, column)
        features.append(feat)
        col += 1
    features = pd.concat(features, axis=1)
    return features

In [ ]:
# Measure the length ofthe epochs
from collections import Counter
e = Counter(data.experiment_traits['learning_epoch'])
ev = [0, e['Pre-Learning'], e['Learning'], e['Post-Learning']]
ev = np.cumsum(ev)
e, ev

In [ ]:
data.experiment_traits.head()

In [ ]:
cf = create_features(cat_features,ord_features,data.experiment_traits)
cf

In [ ]:
p = np.mean(cf.values)
p

#### Init model

In [ ]:
import BayesianHiddenCause as bc
imp.reload(bc)

In [ ]:
bba = bc.BernoulliBetaAssumption(p, 3)

In [ ]:
bba.observe(cf.astype(int).T)

In [ ]:
bba.Gibbs_prepare(5)

In [ ]:
fig = bc.plot_matrix_product('i (observabes)',bba.Z,'Z','t (trials)',bba.Y,'Y','k (causes)',bba.P_x_YZ(),'X estimated')
fig.suptitle('Estimate')
fig = bc.plot_matrix_product('i (observabes)',np.array([[]]),'Z','t (trials)',np.array([[]]),'Y','k (causes)',cf.values.astype(int).T,'X observed')
fig.suptitle('Original')

#### Iterate to get most likely constellations of observed features

In [ ]:
for i in range(0,100):
    bba.Gibbs_iterate()

In [ ]:
links = []
for i in range(0,1000):
    for i in range(0,10):
        bba.Gibbs_iterate()
    links.extend([tuple(col) for col in bba.Z.T])

In [ ]:
from collections import Counter
c = Counter(links)
c

#### Simulate response to pre-defined samples

In [ ]:
def simulate(learner, test_samples, given_i):
    '''Simulate the learners response to the test samples
       taking into account only the features marked true in given_i'''
    # Initialize
    cum, totp = 0, 0
    # Test all possible latent states
    a=([0,1],)*learner.K
    for Y1 in itertools.product(*a):
        Y1 = np.array(Y1)
        # The probability of the given latent state in the model\US
        logpy = learner.logP_y_XZ(Y1, X=test_samples, given_i=given_i)
        py = np.exp(logpy)
        # The Bernoulli parameters for the observables
        px = learner.P_x_YZ(Y=Y1[:,np.newaxis])
        # The animal's response
        behav = px[~given_i]
        #print (Y1, behav)
        # Cumulate
        totp += py
        cum += py * behav
    # The animal's average response for the test samples would be
    return (cum/totp)

In [ ]:
# Give a hint for defining tests
print('The %d features are:\n%s' % (len(cat_features), cat_features))

In [ ]:
# Given variables: all but US
given_i = np.array(map(lambda x: x[0]!='puffed', cat_features))

### Define a well established set of samples where we want to know the behavior

### V0: original protocol questions with current knowledge
#test_samples = None

### V1 (CS+, W+), same, (CS+, W-), same, (CS-, W+), same OR (CS+), same, (CS-), same:
### verify that puffed deosn't count
#if include_w:
#    test_samples = np.array([[1,0,1,1],[1,0,1,0],[1,0,0,1],[1,0,0,0],[0,1,1,0],[0,1,1,1]]).T
#    test_names = ['CS+, W+', 'same', 'CS+, W-', 'same', 'CS-, W+', 'same']
#    test_colors = ['magenta','magenta', 'red','red', 'cyan','cyan', 'lime','lime']
#else:
#    test_samples = np.array([[1,0,1],[1,0,0],[0,1,0],[0,1,1]]).T
#    test_names = ['CS+, W+', 'same', 'CS+, W-', 'same', 'CS-, W+', 'same']
#    test_colors = ['red','red', 'lime','lime']

### V2 (BL, W+), (CS+, W+), (CS+, W-), (CS-, W+), (CS-, W-) OR (BL), (CS+), (CS-)
if include_w:
    test_samples = np.array([[0,0,1,0],[1,0,1,1],[1,0,0,1],[0,1,1,0],[0,1,0,1]]).T
    test_names = ['BL, W+', 'CS+, W+', 'CS+, W-', 'CS-, W+', 'CS-, W-']
    test_colors = ['y', 'magenta', 'red', 'cyan', 'lime'] # ca.short_colors
else:
    test_samples = np.array([[0,0,1],[1,0,1],[0,1,1]]).T
    test_names = ['BL', 'CS+', 'CS-']
    test_colors = ['y', 'red', 'lime']
    
test_samples = pd.DataFrame(test_samples, columns=test_names)

In [ ]:
def do_test(learner, sampler, data, test_samples, given_i, available_from=0, start_trial=None, end_trial=None):
    '''Do forecast using (and changing the state) of learner with the specific sampling
       The interval for training starts at available from and its endpoint slides from start_trial to end_trial.
       The returned time labels correspond to the point where the outcome of the trial is not yet known.'''
    # data: n_features x n_trials
    # test_samples: n_features x n_samples
    # given_i: bool (n_features)
    opinions = [] # np.empty(shape=(0,len(test_samples)))
    expectation = []
    try:
        row_names = data.columns.append([['end']])
    except:
        row_names = None
    try:
        column_names = test_samples.columns
    except:
        column_names = None
    data = np.array(data)
    test_samples = np.array(test_samples)
    
    available_from = int(available_from)
    end_trial = len(data.T) if end_trial is None else int(end_trial)
    start_trial = available_from if start_trial is None else int(start_trial)
    
    assert (available_from >= 0) and (available_from <= start_trial)
    assert (start_trial >= 0) and (start_trial < end_trial)
    assert (end_trial > 0) and (end_trial <= len(data.T))
    
    print('before','<opinions>','<K>')
    # Train learner with first ntrial trials (equivalent weights) and see response
    rows = list(range(start_trial,end_trial+1))
    for available_to in rows:
        learner.clear()
        learner.observe(data[:,available_from:available_to])
        cum, K = 0, 0
        opi, exp = np.zeros((1,test_samples.shape[1])), 0
        for p in sampler(learner):            
            cum += p
            K += p * learner.K
            opi += p * simulate(learner, test_samples, given_i)
            try:
                exp += p * simulate(learner, data[:,[available_to]], given_i)
            except IndexError:
                exp = np.nan
        cum = float(cum)
        opinions.append(opi / cum)
        expectation.append(exp / cum)
        print(available_to, opinions[-1], K / cum)
    index = None if row_names is None else row_names[np.array(rows)]
    opinions = pd.DataFrame(np.row_stack(opinions),
                            index=index, columns=column_names)
    expectation = pd.Series(np.ravel(np.row_stack(expectation)),
                               index=index, name='expect_US').dropna()
    return opinions, expectation

#### Config

In [ ]:
# Decay time of past experiences
decay_time = np.inf
# Start larning at
start = 0 # 0, ev[1]

#### The response according to the last model

In [ ]:
def sample_one(learner):
    learner.Gibbs_prepare(5)
    for i in range(0,100):
        learner.Gibbs_iterate()
    yield 1

learner = bc.BernoulliBetaAssumption(p, 3, decay_time=decay_time)
opinions, expectation = do_test(learner,
        sample_one, cf.astype(int).T, test_samples, given_i, start)

#### The response according to a population of models

In [ ]:
n_models = 50
def sample_Gibbs(learner, n_models):
    learner.Gibbs_prepare(5)
    for i in range(0,100):
        learner.Gibbs_iterate()
    for m in range(0,n_models):
        for i in range(0,10):
            learner.Gibbs_iterate()
        yield 1
        
learner = bc.BernoulliBetaAssumption(p, 3, decay_time=decay_time)
opinions, expectation = do_test(learner,
                                lambda l: sample_Gibbs(l, n_models), cf.astype(int).T,
                                test_samples, given_i, start)

#### The response integrating over all models

#### The response based on randomly chosen matrices

#### Plot

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

class helpmultipage(object):
    def __init__(self, filename):
        self.filename = filename
        self.isopen = False
        self.open()
        
    def __del__(self):
        self.close()
        
    def savefig(self, dpi=None):
        if self.isopen:
            self.pp.savefig(dpi=dpi)

    def open(self):
        if (~self.isopen) and len(self.filename):
            self.pp = PdfPages(self.filename)
            self.isopen = True
        
    def close(self):
        if self.isopen:
            self.pp.close()
        self.isopen = False

In [ ]:
pp = helpmultipage('Bayesian_%s.pdf'%animal)

In [ ]:
from matplotlib.collections import LineCollection
# Plot whether the animal should expect the US based on its previous experiences
t = data['experiment_traits']['idx'].reindex(
    opinions.index).fillna(len(data['experiment_traits']))
fig = plt.figure(figsize=(12,5))
ax = fig.gca()
ax.set_title(animal)
#lines = ax.plot(t,opinions)
# something similar to (conn[line_id][point_id_in_line] is tuple of length 2 or 3 for xyz)
#conn = []
#for time_series in np.array(opinions, ndmin=2).T:
#    conn.append(np.column_stack((t,time_series)))
#line_segments = LineCollection(conn, colors=test_colors)
#ax.add_collection(line_segments)
### finally you get to create an extra Line2D object for all symbols in the legend
for idx, time_series in enumerate(np.array(opinions, ndmin=2).T):
    ax.plot(t, time_series, color=test_colors[idx])
ax.set_xlabel('Trial ID')
ax.set_ylabel('P(expect airpuff)')
ax.vlines(ev-0.5, 0, 1, color='orange')
ax.hlines([0.5], 0, ev[-1], color='grey', linestyles='--')
ax.set_xlim(ev[[0,-1]])
plt.legend(test_names, loc='upper left', title='Ideal learner', fontsize=10)
pp.savefig()

In [ ]:
from matplotlib.collections import LineCollection
mask = behavior['motivation'] # & behavior['sensibility']
mask = mask[mask].index
resp = behavior['fear']
ttype = data['experiment_traits']['context'][mask]
t = data['experiment_traits'].loc[mask,'idx']
fig = plt.figure(figsize=(12,5))
ax = fig.gca()
ax.set_title(animal)
lines1 = ax.scatter(t,resp[mask].astype(int),c='b',marker='s',s=32,label='observed (during trace)')
lines2 = ax.scatter(t,expectation[mask],c='orange',marker='o',s=32,label='calculated (ideal)')
# something similar to (conn[line_id][point_id_in_line] is tuple of length 2 or 3 for xyz)
conn = np.zeros((len(t),2,2))
conn[:,:,0] = t[:,np.newaxis]
conn[:,0,1] = expectation[mask]
conn[:,1,1] = resp[mask].astype(int)
color = ttype.replace('Baseline','y').replace('CS+','magenta').replace('CS-','cyan')
line_segments = LineCollection(conn, colors=color)
ax.add_collection(line_segments)
ax.set_xlabel('Trial ID')
ax.set_ylabel('P(expect airpuff)')
ax.vlines(ev-0.5, -0.1, 1.1, color='orange')
ax.hlines([0.5], 0, ev[-1], color='grey', linestyles='--')
ax.set_xlim(ev[[0,-1]])
ax.set_ylim((-0.1,1.1))
plt.legend(loc='upper left', title='Reference trials with licking', fontsize=10)
pp.savefig()

In [ ]:
pp.close()

#### Convert and save most frequent vectors

In [ ]:
constellations = pd.DataFrame(c, index=[animal]).T
constellations.index.names = [b for a,b in cat_features]
constellations.index = pd.MultiIndex.from_arrays(np.array(constellations.index.tolist()).astype(bool).T,
                                                 names = [b for a,b in cat_features])
constellations

In [ ]:
result = {'constellations':constellations}

In [ ]:
la.store_to_hdf('baydb_'+animal+'.h5', result)